In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon
from shapely.ops import unary_union
from tqdm import tqdm
import json
import gc
import time
from datetime import timedelta

# STEP 3: Connecting Tariffed HS Codes, NAICS Codes and respective CUSMA Non-Utilisation Rates via Concordance Table

In [2]:
tariffed = pd.read_csv('tariff-hscodes.csv', encoding_errors='ignore', dtype={'HS Code': str})

tariffed['HS_Code_6digit'] = (
    tariffed['HS Code']
    .str.replace('.', '', regex=False)
    .str[:6]
)

tariffed = tariffed[['HS_Code_6digit', 'Category']].drop_duplicates()

In [3]:
concordance = pd.read_csv('C616_HS8toNaics6_concord_202505.csv', dtype={'hts10': str})

concordance['HS_Code_6digit'] = (
    concordance['HS8 Code']
    .astype(str)
    .str.zfill(8)
    .str[:6]
)

concordance['HS_Code_2digit'] = (
    concordance['HS8 Code']
    .astype(str)
    .str.zfill(8)
    .str[:2]
)

concordance['naics'] = concordance['NAICS 6 Code'].astype(str)

concordance = concordance[['HS_Code_6digit', 'naics', 'HS_Code_2digit']].drop_duplicates()

In [4]:
util = pd.read_csv('USMCA Utilization Data.csv')

util['HS_Code_2digit'] = (
    util['HS Classification']
    .astype(str)
    .str[:2]
)

util['nonutil_rate'] = util['USMCA_Nonutilisation_May2025']

util = util[['HS_Code_2digit', 'nonutil_rate']]

Setting the non-utilisation rate for those with sectoral tariffs at 1 reflects the fact that the 35% tariffs on non-CUSMA goods does not apply to the sectoral tariffs and that producers impacted by sectoral tariffs cannot use CUSMA to get their goods tariff-free.

In [5]:
naics_tar = concordance.merge(tariffed, on='HS_Code_6digit', how='left')

counts = naics_tar['HS_Code_6digit'].value_counts().reset_index()
naics_tarc = naics_tar.merge(counts, on='HS_Code_6digit', how='left')

naics_imp = naics_tarc.merge(util, on='HS_Code_2digit', how='left')

# Making the codes with a sectoral tariff category assigned to have a non-util rate value of 1
# This ensures that when multiplied later, sectoral tariffs do not affect the CUSMA non-utilisation rates to compute the impact of nonCUSMA 35% tariffs
naics_imp.loc[naics_imp['Category'].notna(), 'nonutil_rate'] = 1
naics_imp

,HS_Code_6digit,naics,HS_Code_2digit,Category,count,nonutil_rate
0,010110,112920,01,NaN,1,0.42
1,010121,112920,01,NaN,1,0.42
2,010129,112920,01,NaN,1,0.42
3,010130,112920,01,NaN,1,0.42
4,010190,112920,01,NaN,1,0.42
...,...,...,...,...,...,...
6969,961620,314990,96,NaN,1,0.84
6970,961700,332439,96,NaN,1,0.84
6971,961800,339990,96,NaN,1,0.84
6972,961900,322291,96,NaN,1,0.84


# STEP 4: Getting Weights by Province/Territory

Since multiple NAICS codes may contribute to the production of one HS code product, and we do not how much of a part does each NAICS contribute to the whole HS code good production, **thus an assumption is made to divide them equally**. Hence, when each export value is added, it is divided them by the count (how many times does that HS Code get repeated)

In [6]:
# Initialize the DataFrame with NAICS data
tariff_exp_val = naics_imp.copy()

# Define all regions to process
regions = ['NL', 'PEI', 'NS', 'NB', 'QC', 'ON', 'MB', 'SK', 'AL', 'BC', 'YK', 'NWT', 'NU']
cols = ['Commodity', 'Value ($)']

for region in regions:
    # Process Global data
    global_df = pd.read_csv(f'{region}-Global.csv', usecols=cols)
    global_df['HS_Code_6digit'] = global_df['Commodity'].str.replace('.', '', regex=False).str[:6]
    global_df[f'{region}_Global'] = global_df['Value ($)']
    global_df = global_df[['HS_Code_6digit', f'{region}_Global']]
    
    tariff_exp_val = tariff_exp_val.merge(global_df, on='HS_Code_6digit', how='left')
    tariff_exp_val[f'{region}_Global'] = tariff_exp_val[f'{region}_Global'] / tariff_exp_val['count']
    
    # Process US data
    us_df = pd.read_csv(f'{region}-US.csv', usecols=cols)
    us_df['HS_Code_6digit'] = us_df['Commodity'].str.replace('.', '', regex=False).str[:6]
    us_df[f'{region}_US'] = us_df['Value ($)']
    us_df = us_df[['HS_Code_6digit', f'{region}_US']]
    
    tariff_exp_val = tariff_exp_val.merge(us_df, on='HS_Code_6digit', how='left')
    tariff_exp_val[f'{region}_US'] = tariff_exp_val[f'{region}_US'] / tariff_exp_val['count']

# Drop the non-relevant columns
tariff_exp_val = tariff_exp_val.drop(columns=['HS_Code_2digit', 'Category', 'count'])

tariff_exp_val = tariff_exp_val.fillna(0)

tariff_exp_val

,HS_Code_6digit,naics,nonutil_rate,NL_Global,NL_US,PEI_Global,PEI_US,NS_Global,NS_US,NB_Global,...,AL_Global,AL_US,BC_Global,BC_US,YK_Global,YK_US,NWT_Global,NWT_US,NU_Global,NU_US
0,010110,112920,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,010121,112920,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,102006.0,102006.0,43608.0,43608.0,0.0,0.0,0.0,0.0,0.0,0.0
2,010129,112920,0.42,0.0,0.0,103341.0,103341.0,767570.0,767570.0,245243.0,...,38107217.0,16300710.0,8940055.0,8863555.0,0.0,0.0,0.0,0.0,0.0,0.0
3,010130,112920,0.42,0.0,0.0,0.0,0.0,0.0,0.0,5591.0,...,0.0,0.0,16453.0,16453.0,0.0,0.0,0.0,0.0,0.0,0.0
4,010190,112920,0.42,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6969,961620,314990,0.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,26374.0,23452.0,0.0,0.0,0.0,0.0,0.0,0.0
6970,961700,332439,0.84,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,4454.0,4454.0,191510.0,69403.0,0.0,0.0,0.0,0.0,0.0,0.0
6971,961800,339990,0.84,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,57388.0,36336.0,0.0,0.0,0.0,0.0,0.0,0.0
6972,961900,322291,0.84,0.0,0.0,0.0,0.0,0.0,0.0,113029441.0,...,10736.0,0.0,408726.0,5790.0,0.0,0.0,0.0,0.0,0.0,0.0


The formula to calculate the weights is a ratio of US exports to Global exports, which is then multiplied by the respective CUSMA non-utilisation rate.  
  
We are taking the assumption that **the non-utilisation rate for each NAICS code is set at the mean, and it is not weighted**. This is because non-utilisation rate is first given to HS Codes, and each NAICS industry can produce products across different HS Codes and therefore each of their product exports may have different non-utilisation rates.

In [7]:
# Step 1: Aggregate ONLY the necessary sums (Global/US columns)
aggregates = {
    **{f'{region}_Global': (f'{region}_Global', 'sum') for region in regions},
    **{f'{region}_US': (f'{region}_US', 'sum') for region in regions},
    'nonutil_rate':   ('nonutil_rate', 'mean')
}

weight_naics = tariff_exp_val.groupby('naics', as_index=False).agg(**aggregates)

# Step 2: Compute rates and keep ONLY those columns
for region in regions:
    global_col = f'{region}_Global'
    us_col = f'{region}_US'
    rate_col = f'{region}_Rate'
    
    weight_naics[rate_col] = (
        weight_naics[us_col] / weight_naics[global_col] * weight_naics['nonutil_rate']
    )

# Step 3: Select only naics, naics_2digit, and rate columns
final_columns = ['naics'] + [f'{region}_Rate' for region in regions]
weight_naics = weight_naics[final_columns]
weight_naics = weight_naics.fillna(0)

# Result
weight_naics

,naics,NL_Rate,PEI_Rate,NS_Rate,NB_Rate,QC_Rate,ON_Rate,MB_Rate,SK_Rate,AL_Rate,BC_Rate,YK_Rate,NWT_Rate,NU_Rate
0,111110,0.000000,0.000000,0.000000,0.620000,0.016695,0.057076,0.064938,0.000728,0.024138,0.000000,0.000000,0.0,0.000000
1,111120,0.678462,0.678462,0.000000,0.136710,0.043700,0.406677,0.103250,0.052447,0.037429,0.031443,0.000000,0.0,0.000000
2,111130,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,111140,0.000000,0.000000,0.405714,0.405714,0.042229,0.033932,0.032811,0.042095,0.035477,0.031522,0.000000,0.0,0.000000
4,111150,0.000000,0.000000,0.000000,0.000000,0.194542,0.074227,0.370000,0.000000,0.370000,0.243865,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,339920,0.481951,0.296708,0.536874,0.149737,0.484751,0.417990,0.457369,0.482840,0.531677,0.409661,0.548929,0.0,0.000000
306,339930,0.337158,0.000000,0.372909,0.394500,0.326389,0.233975,0.316760,0.384623,0.346251,0.241962,0.000000,0.0,0.394500
307,339940,0.000000,0.000000,0.048109,0.804283,0.498074,0.731903,0.809208,0.812800,0.701242,0.726585,0.000000,0.0,0.000000
308,339950,0.000000,0.668000,0.667704,0.642297,0.651382,0.639883,0.663664,0.668000,0.470009,0.598870,0.668000,0.0,0.668000


# STEP 5: Using filtered NAICS Codes to filter directly-impacted businesses and estimate directly-impacted employees

In [8]:
auto_naics = set(naics_imp[naics_imp['Category'] == 'Auto']['naics'].unique())
alum_naics = set(naics_imp[naics_imp['Category'] == 'Aluminum']['naics'].unique())
steel_naics = set(naics_imp[naics_imp['Category'] == 'Steel']['naics'].unique())
copper_naics = set(naics_imp[naics_imp['Category'] == 'Copper']['naics'].unique())
lum_naics = set(naics_imp[naics_imp['Category'] == 'Lumber']['naics'].unique())
ene_naics = set(naics_imp[naics_imp['Category'] == 'Energy Mineral']['naics'].unique())
noncusma_naics = set(naics_imp[naics_imp['Category'].isna()]['naics'].unique())
total_naics = set(naics_imp['naics'].unique())

In [9]:
col_i = ['DA',
        'All_Businesses', 'All_Employees',
        'Auto_B', 'Auto_E',
        'Alum_B', 'Alum_E',
        'Steel_B', 'Steel_E',
        'Cop_B', 'Cop_E',
        'Lum_B', 'Lum_E',
        'Ene_B', 'Ene_E',
        'CUSMA_B', 'CUSMA_E',
        'Total_B', 'Total_E',
        #add columns here, column titles are every value of total_naics
       ]

# Add individual NAICS codes as column headers for Est_Employees by NAICS
col_i.extend(sorted(total_naics))

business = pd.DataFrame(columns = col_i)

In [10]:
chunk_size = 1_000_000  # Start with 100k rows per chunk
results = []  # Store aggregated results

province_code = {
    10: 'NL',
    11: 'PEI',
    12: 'NS',
    13: 'NB',
    24: 'QC',
    35: 'ON',
    46: 'MB',
    47: 'SK',
    48: 'AL',
    59: 'BC',
    60: 'YK',
    61: 'NWT',
    62: 'NU'
}

# Remove unwanted columns
cols_to_keep = [col for col in pd.read_csv('Dec2022_Estabcounts_byDA.csv', encoding='ISO-8859-1', nrows=1).columns 
               if col != 'Without employees']

# ✅ Start total timer
total_start = time.time()

# ✅ Track chunk count
chunk_num = 0

# Read in chunks
for chunk in pd.read_csv('Dec2022_Estabcounts_byDA.csv', encoding='ISO-8859-1', chunksize=chunk_size, usecols=cols_to_keep):

    chunk_num += 1
    chunk_start = time.time()  # ✅ start chunk timer
    
    # Step 1: Filter unwanted rows
    chunk = chunk[~chunk['NAICS'].isin(['Sub-total, classified', 'Unclassified', 'Total'])]
    
    # Step 2: Clean/transform data
    chunk['NAICS'] = chunk['NAICS'].astype(str).str[:6]
    chunk['Business_per_NAICS'] = chunk['Total, with employees']
    chunk['Est_Employees'] = (
        chunk['1-4'] * 3 +
        chunk['5-9'] * 7 +
        chunk['10-19'] * 15 +
        chunk['20-49'] * 35 +
        chunk['50-99'] * 75 +
        chunk['100-199'] * 150 +
        chunk['200-499'] * 350 +
        chunk['500 +'] * 550
    )
    chunk['ProvinceCode'] = chunk['DisseminationAre'].astype(str).str[:2].astype(int)
    chunk['Province'] = chunk['ProvinceCode'].map(province_code)

    # Step 3: Merge with weight_naics to get weights
    merged = chunk.merge(weight_naics, how='left', left_on='NAICS', right_on='naics')

    # Step 4: Lookup province rate per row
    merged['Rate'] = merged.apply(lambda row: row.get(f"{row['Province']}_Rate", 1.0), axis=1)
    merged['Weighted_Business'] = np.ceil(merged['Business_per_NAICS'] * merged['Rate'])
    merged['Weighted_Employees'] = np.ceil(merged['Est_Employees'] * merged['Rate'])

    # Step 5: Group and aggregate
    for da, group in merged.groupby('DisseminationAre'):
        
        result = {
            'DA': da,
            'All_Businesses': group['Business_per_NAICS'].sum(),
            'All_Employees': group['Est_Employees'].sum(),
            'Auto_B': group.loc[group['NAICS'].isin(auto_naics), 'Weighted_Business'].sum(),
            'Auto_E': group.loc[group['NAICS'].isin(auto_naics), 'Weighted_Employees'].sum(),
            'Alum_B': group.loc[group['NAICS'].isin(alum_naics), 'Weighted_Business'].sum(),
            'Alum_E': group.loc[group['NAICS'].isin(alum_naics), 'Weighted_Employees'].sum(),
            'Steel_B': group.loc[group['NAICS'].isin(steel_naics), 'Weighted_Business'].sum(),
            'Steel_E': group.loc[group['NAICS'].isin(steel_naics), 'Weighted_Employees'].sum(),
            'Cop_B': group.loc[group['NAICS'].isin(copper_naics), 'Weighted_Business'].sum(),
            'Cop_E': group.loc[group['NAICS'].isin(copper_naics), 'Weighted_Employees'].sum(),
            'Lum_B': group.loc[group['NAICS'].isin(lum_naics), 'Weighted_Business'].sum(),
            'Lum_E': group.loc[group['NAICS'].isin(lum_naics), 'Weighted_Employees'].sum(),
            'Ene_B': group.loc[group['NAICS'].isin(ene_naics), 'Weighted_Business'].sum(),
            'Ene_E': group.loc[group['NAICS'].isin(ene_naics), 'Weighted_Employees'].sum(),
            'CUSMA_B': group.loc[group['NAICS'].isin(noncusma_naics), 'Weighted_Business'].sum(),
            'CUSMA_E': group.loc[group['NAICS'].isin(noncusma_naics), 'Weighted_Employees'].sum(),
            'Total_B': group.loc[group['NAICS'].isin(total_naics), 'Weighted_Business'].sum(),
            'Total_E': group.loc[group['NAICS'].isin(total_naics), 'Weighted_Employees'].sum()
        }

        # Add per-NAICS Est_Employees values
        for naics in total_naics:
            result[naics] = group.loc[group['NAICS'] == naics, 'Est_Employees'].sum()

        results.append(result)
        
    # ✅ End chunk timer
    chunk_time = time.time() - chunk_start
    print(f"Chunk {chunk_num} processed in {chunk_time:.2f} seconds")

# ✅ End total timer
total_time = time.time() - total_start
print(f"\n✅ All chunks processed in {total_time:.2f} seconds.")

# Combine all chunk results
business = pd.DataFrame(results).groupby('DA', as_index=False).sum()
business

Chunk 1 processed in 161.32 seconds
Chunk 2 processed in 143.50 seconds
Chunk 3 processed in 145.32 seconds
Chunk 4 processed in 139.92 seconds
Chunk 5 processed in 135.22 seconds
Chunk 6 processed in 132.87 seconds
Chunk 7 processed in 134.25 seconds
Chunk 8 processed in 134.69 seconds
Chunk 9 processed in 133.66 seconds
Chunk 10 processed in 133.06 seconds
Chunk 11 processed in 133.74 seconds
Chunk 12 processed in 133.08 seconds
Chunk 13 processed in 133.52 seconds
Chunk 14 processed in 134.54 seconds
Chunk 15 processed in 133.71 seconds
Chunk 16 processed in 133.15 seconds
Chunk 17 processed in 133.37 seconds
Chunk 18 processed in 133.21 seconds
Chunk 19 processed in 133.57 seconds
Chunk 20 processed in 133.78 seconds
Chunk 21 processed in 133.36 seconds
Chunk 22 processed in 133.32 seconds
Chunk 23 processed in 133.81 seconds
Chunk 24 processed in 133.76 seconds
Chunk 25 processed in 133.11 seconds
Chunk 26 processed in 134.43 seconds
Chunk 27 processed in 133.86 seconds
Chunk 28 p

,DA,All_Businesses,All_Employees,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,...,325314,327120,114114,212394,324110,331317,325120,315990,334410,111120
0,10000000,170,1006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,10010165,22,266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,10010166,2,6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,10010167,6,22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,10010168,5,19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55246,62080023,2,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55247,62080024,11,113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55248,62080025,11,356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
55249,62080026,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


# STEP 6: Regrouping filtered data into ADAs

In [11]:
da = gpd.read_file('lda_000b21a_e.shp')
da ['DA'] = da ['DAUID']
da ['DADGUID'] = da ['DGUID']
da = da[['DA', 'DADGUID']]

ada = gpd.read_file('lada000b21a_e.shp')
adac = ada.copy()
adac ['ADADGUID'] = adac ['DGUID']
adac = adac[['ADADGUID', 'geometry']]

relation = pd.read_csv('ada_da_relation.csv')

In [12]:
da_relation = da.merge(relation, on='DADGUID', how='left')
full_relation = da_relation.merge(adac, on='ADADGUID', how='left')
full_relation['DA'] = pd.to_numeric(full_relation['DA'], errors='coerce').astype('Int64')

In [13]:
business_merged =(
    business.merge(full_relation, on='DA', how='right')
    .fillna(0)
    .astype({col: 'int64' for col in business.columns if col != 'DA'})
) 

# Group by ADADGUID and aggregate
agg_dict = {
    'All_Businesses': ('All_Businesses', 'sum'),
    'All_Employees': ('All_Employees', 'sum'),
    'Auto_B': ('Auto_B', 'sum'),
    'Auto_E': ('Auto_E', 'sum'),
    'Alum_B': ('Alum_B', 'sum'),
    'Alum_E': ('Alum_E', 'sum'),
    'Steel_B': ('Steel_B', 'sum'),
    'Steel_E': ('Steel_E', 'sum'),
    'Cop_B': ('Cop_B', 'sum'),
    'Cop_E': ('Cop_E', 'sum'),
    'Lum_B': ('Lum_B', 'sum'),
    'Lum_E': ('Lum_E', 'sum'),
    'Ene_B': ('Ene_B', 'sum'),
    'Ene_E': ('Ene_E', 'sum'),
    'CUSMA_B': ('CUSMA_B', 'sum'),
    'CUSMA_E': ('CUSMA_E', 'sum'),
    'Total_B': ('Total_B', 'sum'),
    'Total_E': ('Total_E', 'sum'),
    'geometry': ('geometry', 'first')
}

# Add dynamic aggregation rules for each NAICS code
for naics in total_naics:
    agg_dict[naics] = (naics, 'sum')

# Perform grouped aggregation
business_grouped = business_merged.groupby('ADADGUID', as_index=False).agg(**agg_dict)

business_grouped

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\3982846873.py:35: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  business_grouped = business_merged.groupby('ADADGUID', as_index=False).agg(**agg_dict)


,ADADGUID,All_Businesses,All_Employees,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,...,325314,327120,114114,212394,324110,331317,325120,315990,334410,111120
0,2021S051610010001,219,2910,0,0,1,1,1,1,0,...,0,0,10,0,0,0,0,0,0,0
1,2021S051610010002,66,394,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2021S051610010003,288,4186,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,2021S051610010004,438,9064,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,2021S051610010005,203,1441,2,5,1,3,1,3,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,11,356,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5429,2021S051662080005,13,414,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5430,2021S051662080006,8,295,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# STEP 7: Processing it into Centroids for Counts and Choropleth for Rates

Since the earlier table shows the *weighted* numbers of directly exposed businesses and employees (by work location) together with *total* number of employees (by work location) for each affected NAICS code, the former is separated from the latter. The former needs to be processed into separate GDFs for centroids (to show counts) and choropleths (to show percentages)

In [14]:
excluded_cols = [
    'All_Businesses', 'All_Employees',
    'Auto_B', 'Auto_E', 'Alum_B', 'Alum_E',
    'Steel_B', 'Steel_E', 'Cop_B', 'Cop_E',
    'Lum_B', 'Lum_E', 'Ene_B', 'Ene_E', 
    'CUSMA_B', 'CUSMA_E', 'Total_B', 'Total_E', 
    'geometry'
]

business_filter = business_grouped[['ADADGUID'] + [col for col in business_grouped.columns if col in excluded_cols]]

business_census = business_grouped[[col for col in business_grouped.columns if col not in excluded_cols]]

In [15]:
# Convert to GeoDataFrame
cent_gdf = gpd.GeoDataFrame(business_filter, geometry='geometry', crs = 'EPSG:3347')

# Set a point within each polygon
cent_gdf = cent_gdf.drop(columns=['All_Businesses', 'All_Employees'])
cent_gdf['geometry'] = cent_gdf.geometry.representative_point()
cent_gdf.set_geometry('geometry', inplace=True)
cent_gdf

,ADADGUID,Auto_B,Auto_E,Alum_B,Alum_E,Steel_B,Steel_E,Cop_B,Cop_E,Lum_B,Lum_E,Ene_B,Ene_E,CUSMA_B,CUSMA_E,Total_B,Total_E,geometry
0,2021S051610010001,0,0,1,1,1,1,0,0,2,6,0,0,16,791,16,791,POINT (8927316.642 2156398.591)
1,2021S051610010002,0,0,0,0,0,0,0,0,0,0,0,0,2,10,2,10,POINT (8966678.602 2164982.461)
2,2021S051610010003,0,0,1,1,1,1,0,0,4,37,0,0,9,49,9,49,POINT (8934606.046 2144277.889)
3,2021S051610010004,0,0,1,1,1,1,1,1,0,0,2,25,7,67,7,67,POINT (8976138.327 2157799.904)
4,2021S051610010005,2,5,1,3,1,3,0,0,0,0,0,0,5,29,5,29,POINT (8970965.684 2157632.7)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (5259010.983 3653721.631)
5429,2021S051662080005,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (6041286.124 3573109.546)
5430,2021S051662080006,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (6281761.356 3557612.071)
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,POINT (5550011.303 3546406.473)


In [16]:
choro_cols = business_filter.copy()

tars = ['Auto', 'Alum', 'Steel', 'Cop', 'Lum', 'Ene', 'CUSMA', 'Total']

for tar in tars:
    tar_1 = f'{tar}_1'
    tar_2 = f'{tar}_2'

    choro_cols[tar_1] = (
        choro_cols[f'{tar}_B']/choro_cols['All_Businesses']
    )

    choro_cols[tar_2] = (
        choro_cols[f'{tar}_E']/choro_cols['All_Employees']
    )

choro_cols = choro_cols[['ADADGUID', 'All_Businesses', 'All_Employees'] + [f'{tar}_1' for tar in tars] + [f'{tar}_2' for tar in tars] + ['geometry']]
choro_cols = choro_cols.fillna(0)

choro_gdf = gpd.GeoDataFrame(choro_cols, geometry = 'geometry', crs = 'EPSG:3347')
choro_gdf

,ADADGUID,All_Businesses,All_Employees,Auto_1,Alum_1,Steel_1,Cop_1,Lum_1,Ene_1,CUSMA_1,Total_1,Auto_2,Alum_2,Steel_2,Cop_2,Lum_2,Ene_2,CUSMA_2,Total_2,geometry
0,2021S051610010001,219,2910,0.000000,0.004566,0.004566,0.000000,0.009132,0.000000,0.073059,0.073059,0.00000,0.000344,0.000344,0.00000,0.002062,0.000000,0.271821,0.271821,"MULTIPOLYGON (((8921559.157 2130255.903, 89215..."
1,2021S051610010002,66,394,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030303,0.030303,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.025381,0.025381,"POLYGON ((8959571.943 2171799.486, 8959576.689..."
2,2021S051610010003,288,4186,0.000000,0.003472,0.003472,0.000000,0.013889,0.000000,0.031250,0.031250,0.00000,0.000239,0.000239,0.00000,0.008839,0.000000,0.011706,0.011706,"MULTIPOLYGON (((8938088.457 2157739.16, 893808..."
3,2021S051610010004,438,9064,0.000000,0.002283,0.002283,0.002283,0.000000,0.004566,0.015982,0.015982,0.00000,0.000110,0.000110,0.00011,0.000000,0.002758,0.007392,0.007392,"POLYGON ((8976008.48 2163749.357, 8976015.274 ..."
4,2021S051610010005,203,1441,0.009852,0.004926,0.004926,0.000000,0.000000,0.000000,0.024631,0.024631,0.00347,0.002082,0.002082,0.00000,0.000000,0.000000,0.020125,0.020125,"POLYGON ((8969716.249 2163377.051, 8969785.883..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,11,356,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((5263454.031 3662224.177, 52634..."
5429,2021S051662080005,13,414,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((6043125.089 3568426.163, 60431..."
5430,2021S051662080006,8,295,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((6280121.289 3558103.571, 62801..."
5431,2021S051662080007,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((5544792.883 3549525.897, 55447..."


# STEP 8: Generating Weight of Directly Exposed Jobs to Total Accessible Jobs in Each Industry

It is likely that while employees live close to their workplace, they do not live in the same ADA as they work in.  
  
StatsCan Census 2021 data shows a huge drop in the number of Canadians who travel more than 15km to their work vis-a-vis those who travel less than that distance to work.  
  
Thus, this cell creates a dictionary where for each ADA, it lists down, including itself, the ADA IDs within a 15km buffer around it (for small ADAs) or ADA IDs that are adjacent to it (for large ADAs). Small ADAs are defined as ADAs with an area less than (15km)^2.

In [17]:
# Copy from earlier ADA shapefile and ensure it's in projected CRS (EPSG:3347)
adas = ada.to_crs("EPSG:3347")

# Create a column to mark if ADA is "small" (<= 706 km2)
adas['is_small'] = adas['LANDAREA'] <= 706

# Build spatial index once
adas_sindex = adas.sindex

# Simplify geometries early to reduce memory use
adas['geometry'] = adas['geometry'].simplify(100)

# Prepare empty dictionary
ada_neighbors = {}

# Process in chunks to avoid RAM overload
chunk_size = 500
n = len(adas)

# Track overall time
start_time = time.time()

for start in tqdm(range(0, n, chunk_size)):
    chunk_start_time = time.time()
    end = min(start + chunk_size, n)
    chunk = adas.iloc[start:end].copy()

    for idx, row in chunk.iterrows():
        ada_uid = row['DGUID']
        geom = row['geometry']
        is_small = row['is_small']

        if is_small:
            buffer_geom = geom.buffer(15000)
            possible_matches_index = list(adas_sindex.intersection(buffer_geom.bounds))
            possible_matches = adas.iloc[possible_matches_index]
            matches = possible_matches[possible_matches.geometry.intersects(buffer_geom)]
        else:
            possible_matches_index = list(adas_sindex.intersection(geom.bounds))
            possible_matches = adas.iloc[possible_matches_index]
            matches = possible_matches[possible_matches.geometry.touches(geom) | (possible_matches['DGUID'] == ada_uid)]

        ada_neighbors[ada_uid] = matches['DGUID'].tolist()

    del chunk
    gc.collect()

    # Print chunk timing
    chunk_elapsed = time.time() - chunk_start_time
    print(f"Chunk {start}-{end} processed in {timedelta(seconds=chunk_elapsed)}")

# Overall timing
total_elapsed = time.time() - start_time
print(f"\nTotal time taken: {timedelta(seconds=total_elapsed)}")

# Optionally save the dictionary to disk
with open("ada_neighbors.json", "w") as f:
    json.dump(ada_neighbors, f)

  9%|███████▌                                                                           | 1/11 [00:24<04:02, 24.28s/it]

Chunk 0-500 processed in 0:00:24.280664


 18%|███████████████                                                                    | 2/11 [00:26<01:40, 11.13s/it]

Chunk 500-1000 processed in 0:00:01.918972


 27%|██████████████████████▋                                                            | 3/11 [00:28<00:57,  7.13s/it]

...

Chunk 4000-4500 processed in 0:00:01.297525


 91%|██████████████████████████████████████████████████████████████████████████▌       | 10/11 [01:51<00:07,  7.58s/it]

Chunk 4500-5000 processed in 0:00:05.797197


100%|██████████████████████████████████████████████████████████████████████████████████| 11/11 [03:45<00:00, 20.48s/it]

Chunk 5000-5433 processed in 0:01:53.765980

Total time taken: 0:03:45.293965


The dictionary is then used in conjunction with the *total* count of employees (by work location), as separated in Cell 13 above, to find out the likely number of jobs of each 6-digit NAICS, and total number of jobs, that are 'accessible' from each ADA --> going by the assumption of travel distance made by Canadians to go to work from Census 2021 data

In [18]:
# Ensure 'ADADGUID' is the index for fast lookup
business_census_indexed = business_census.set_index('ADADGUID')

# Prepare list to collect results
aggregated_results = []

# Loop through ADA + its neighbors
for ada_id, neighbor_list in tqdm(ada_neighbors.items()):
    # Filter business_census rows for all neighbors
    rows = business_census_indexed.loc[business_census_indexed.index.intersection(neighbor_list)]
    
    # Sum across all rows (by column)
    summed = rows.sum()
    
    # Store result with ADA ID
    result = summed.to_dict()
    result['ADADGUID'] = ada_id
    
    aggregated_results.append(result)

# Convert to DataFrame
jobs = pd.DataFrame(aggregated_results)

100%|████████████████████████████████████████████████████████████████████████████| 5433/5433 [00:04<00:00, 1090.66it/s]


This is to find out the rate of directly exposed jobs (6-digit NAICS) to total jobs in each industry (1-digit NAICS jobs) that are accessible from each ADA

In [19]:
jobs_rate = jobs.copy()

cola = [col for col in jobs.columns if col != 'ADADGUID']

# Calculate summed groups by first digit of column name
jobs_rate['Sum1'] = jobs_rate[[col for col in cola if col.startswith('1')]].sum(axis=1)
jobs_rate['Sum2'] = jobs_rate[[col for col in cola if col.startswith('2')]].sum(axis=1)
jobs_rate['Sum3'] = jobs_rate[[col for col in cola if col.startswith('3')]].sum(axis=1)

# Compute share per column
for col in cola:
    col_rate = f'{col}_R'
    if col.startswith('1'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum1']
    elif col.startswith('2'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum2']
    elif col.startswith('3'):
        jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
    else:
        jobs_rate[col_rate] = 0  # fallback in case of unexpected prefix

# Final filtered DataFrame: only ADADGUID and the *_R columns
jobs_rate = jobs_rate[['ADADGUID'] + [f'{col}_R' for col in cola]]

jobs_rate = jobs_rate.fillna(0)

jobs_rate

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\3702249403.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\3702249403.py:18: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  jobs_rate[col_rate] = jobs_rate[col] / jobs_rate['Sum3']
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\3702249403.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which 

,ADADGUID,333920_R,327390_R,332329_R,326111_R,111419_R,312220_R,321991_R,325910_R,327990_R,...,325314_R,327120_R,114114_R,212394_R,324110_R,331317_R,325120_R,315990_R,334410_R,111120_R
0,2021S051610010001,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.059524,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
1,2021S051610010002,0.005297,0.0,0.002472,0.000000,0.004127,0.0,0.0,0.0,0.002472,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
2,2021S051610010003,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.025974,0.0,0.160303,0.0,0.00204,0.0,0.0,0.0
3,2021S051610010004,0.003981,0.0,0.001858,0.003981,0.003713,0.0,0.0,0.0,0.001858,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
4,2021S051610010005,0.003981,0.0,0.001858,0.003981,0.003713,0.0,0.0,0.0,0.001858,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
5429,2021S051662080005,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
5430,2021S051662080006,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0
5431,2021S051662080007,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.0,0.0


# STEP 9: Applying Jobs Weight to Census Data

Census 2021 Data reports residents' occupational NAICS code at the two-digit level

In [20]:
census = pd.read_csv('98-401-X2021012_English_CSV_data.csv', encoding='latin1')

census = census[census['CHARACTERISTIC_ID'].isin([2259, 2262, 2263, 2266])] # Only taking the relevant 2-digit NAICS codes from Census 2021 data

census['ADADGUID'] = census['DGUID']

census['ProvinceCode'] = census['ADADGUID'].str[9:11].astype(int)

census['Province'] = census['ProvinceCode'].map(province_code)

census['CHARACTERISTIC_NAME'] = (
    census['CHARACTERISTIC_NAME']
    .str.replace(' ', '', regex=False)
    .str[:2]
)

census = census[['ADADGUID', 'ProvinceCode', 'Province', 'CHARACTERISTIC_NAME', 'C1_COUNT_TOTAL']]

census_pivot = census.pivot_table(
    index=['ADADGUID', 'ProvinceCode', 'Province'],
    columns='CHARACTERISTIC_NAME',
    values='C1_COUNT_TOTAL',
    aggfunc='first'
).reset_index()

census_pivot

CHARACTERISTIC_NAME,ADADGUID,ProvinceCode,Province,11,21,31,To
0,2021S051610010001,10,NL,455.0,120.0,665.0,3715.0
1,2021S051610010002,10,NL,50.0,70.0,60.0,2120.0
2,2021S051610010003,10,NL,160.0,100.0,310.0,4530.0
3,2021S051610010004,10,NL,20.0,170.0,130.0,5145.0
4,2021S051610010005,10,NL,35.0,215.0,130.0,5190.0
...,...,...,...,...,...,...,...
4957,2021S051662080002,62,NU,10.0,15.0,10.0,860.0
4958,2021S051662080003,62,NU,0.0,0.0,0.0,210.0
4959,2021S051662080004,62,NU,10.0,0.0,0.0,510.0
4960,2021S051662080005,62,NU,10.0,10.0,0.0,465.0


Thus the weights from Step 8 is used to estimate how many employees (by primary residence) are working in industries directly exposed to tariffs

In [21]:
# Merge on ADADGUID to align both datasets
merged = census_pivot.merge(jobs_rate, on='ADADGUID', how='right')  # or 'left' if census is base

# Start building the adjusted DataFrame
adjusted_jobs = merged[['ADADGUID', 'Province', 'To']].copy()

# Compute adjusted values
for col in cola:
    rate_col = f'{col}_R'
    prefix = col[:2]

    if prefix in ['21', '22']:
        source_col = '21'
    elif prefix in ['31', '32', '33']:
        source_col = '31'
    else:
        source_col = prefix

    adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])

adjusted_jobs['Sum'] = adjusted_jobs.drop(columns=['ADADGUID', 'Province', 'To']).sum(axis=1)

adjusted_jobs = adjusted_jobs.fillna(0)

adjusted_jobs['Province'] = adjusted_jobs['Province'].mask(
    adjusted_jobs['Province'].isin([0, np.nan]),
    adjusted_jobs['ADADGUID'].str[9:11].astype(int).map(province_code)
)

adjusted_jobs

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  adjusted_jobs[col] = np.ceil(merged[rate_col] * merged[source_col])
C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\2488107975.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

,ADADGUID,Province,To,333920,327390,332329,326111,111419,312220,321991,...,327120,114114,212394,324110,331317,325120,315990,334410,111120,Sum
0,2021S051610010001,NL,3715.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,28.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1129.0
1,2021S051610010002,NL,2120.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,225.0
2,2021S051610010003,NL,4530.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,50.0,0.0,1.0,0.0,0.0,0.0,590.0
3,2021S051610010004,NL,5145.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,379.0
4,2021S051610010005,NL,5190.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,437.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,NU,510.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5429,2021S051662080005,NU,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5430,2021S051662080006,NU,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5431,2021S051662080007,NU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# STEP 10: Applying Export Weights to the Census Data

Export weights from Step 4 are applied to account for regional differences

In [22]:
# Step 1: Melt to long format
long_weight = weight_naics.melt(id_vars='naics', var_name='ProvinceRate', value_name='Rate')

long_weight['naics'] = long_weight['naics'].astype(str) + '_r'

long_weight['Province'] = long_weight['ProvinceRate'].str[:2]

# Step 2: Pivot to wide format
weight_naics_pivot = long_weight.pivot(index=['Province', 'ProvinceRate'], columns='naics', values='Rate').reset_index()

weight_naics_pivot

naics,Province,ProvinceRate,111110_r,111120_r,111130_r,111140_r,111150_r,111160_r,111190_r,111211_r,...,337215_r,337910_r,337920_r,339110_r,339910_r,339920_r,339930_r,339940_r,339950_r,339990_r
0,AL,AL_Rate,0.024138,0.037429,0.0,0.035477,0.370000,0.000000,0.117974,0.0,...,0.845708,0.514049,0.190000,0.459226,0.443710,0.531677,0.346251,0.701242,0.470009,0.469975
1,BC,BC_Rate,0.000000,0.031443,0.0,0.031522,0.243865,0.000000,0.156439,0.0,...,0.760972,0.186107,0.189876,0.538893,0.633485,0.409661,0.241962,0.726585,0.598870,0.349176
2,MB,MB_Rate,0.064938,0.103250,0.0,0.032811,0.370000,0.000000,0.264963,0.0,...,0.851964,0.790000,0.175574,0.450404,0.258747,0.457369,0.316760,0.809208,0.663664,0.574522
3,NB,NB_Rate,0.620000,0.136710,0.0,0.405714,0.000000,0.000000,0.000000,0.0,...,0.853178,0.790000,0.162405,0.741850,0.767692,0.149737,0.394500,0.804283,0.642297,0.498666
4,NL,NL_Rate,0.000000,0.678462,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.818052,0.000000,0.000000,0.360563,0.231186,0.481951,0.337158,0.000000,0.000000,0.106638
5,NS,NS_Rate,0.000000,0.000000,0.0,0.405714,0.000000,0.000000,0.000000,0.0,...,0.792772,0.000000,0.000000,0.608397,0.634471,0.536874,0.372909,0.048109,0.667704,0.545332
6,NU,NU_Rate,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.752273,0.767692,0.000000,0.394500,0.000000,0.668000,0.683868
7,NW,NWT_Rate,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.767692,0.000000,0.000000,0.000000,0.000000,0.000000
8,ON,ON_Rate,0.057076,0.406677,0.0,0.033932,0.074227,0.315716,0.206290,0.0,...,0.824980,0.728146,0.174355,0.526610,0.700647,0.417990,0.233975,0.731903,0.639883,0.608112
9,PE,PEI_Rate,0.000000,0.678462,0.0,0.000000,0.000000,0.000000,0.370000,0.0,...,0.095587,0.000000,0.000000,0.000000,0.767692,0.296708,0.000000,0.000000,0.668000,0.670449


In [23]:
# Step 1: Merge adjusted_jobs with weight_naics_pivot on Province
census_byjobs = adjusted_jobs.merge(weight_naics_pivot, on='Province', how='left')

# Step 2: Multiply each column by its corresponding rate
for col in cola:
    rate_col = f'{col}_r'
    
    census_byjobs[col] = np.ceil(census_byjobs[col] * census_byjobs[rate_col])

# Step 3: Keep only ADADGUID and updated values
census_byjobs = census_byjobs[['ADADGUID', 'To'] + cola]

# Define output dictionary
grouped_data = {
    'ADADGUID': census_byjobs['ADADGUID'],  # retain ADA ID
    'Census': census_byjobs['To'],
    'Auto_C': census_byjobs[[col for col in cola if col in auto_naics]].sum(axis=1),
    'Alum_C': census_byjobs[[col for col in cola if col in alum_naics]].sum(axis=1),
    'Steel_C': census_byjobs[[col for col in cola if col in steel_naics]].sum(axis=1),
    'Cop_C': census_byjobs[[col for col in cola if col in copper_naics]].sum(axis=1),
    'Lum_C': census_byjobs[[col for col in cola if col in lum_naics]].sum(axis=1),
    'Ene_C': census_byjobs[[col for col in cola if col in ene_naics]].sum(axis=1),
    'CUSMA_C': census_byjobs[[col for col in cola if col in noncusma_naics]].sum(axis=1),
    'Total_C': census_byjobs.iloc[:, 2:].sum(axis=1)
}

# Create final grouped DataFrame
census_bytariffs = pd.DataFrame(grouped_data)
census_bytariffs

,ADADGUID,Census,Auto_C,Alum_C,Steel_C,Cop_C,Lum_C,Ene_C,CUSMA_C,Total_C
0,2021S051610010001,3715.0,0.0,1.0,1.0,0.0,5.0,0.0,579.0,579.0
1,2021S051610010002,2120.0,6.0,8.0,8.0,1.0,3.0,12.0,86.0,86.0
2,2021S051610010003,4530.0,0.0,1.0,2.0,0.0,8.0,36.0,258.0,258.0
3,2021S051610010004,5145.0,6.0,9.0,8.0,1.0,4.0,24.0,107.0,107.0
4,2021S051610010005,5190.0,6.0,9.0,8.0,1.0,4.0,29.0,122.0,122.0
...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,510.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5429,2021S051662080005,465.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5430,2021S051662080006,280.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5431,2021S051662080007,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# STEP 11: Processing and adding the data to Choropleth and Centroid GDFs

Add the data on employees (by primary residence) to the GDFs produced in Step 7

In [24]:
centroids = cent_gdf.merge(census_bytariffs, on='ADADGUID', how='left')
centroids = centroids[['ADADGUID'] + [f'{tar}_B' for tar in tars] + [f'{tar}_E' for tar in tars] + [f'{tar}_C' for tar in tars] + ['geometry']]
centroids = centroids.to_crs('EPSG:4326')
centroids.to_file('centroids.geojson', driver='GeoJSON')
centroids

,ADADGUID,Auto_B,Alum_B,Steel_B,Cop_B,Lum_B,Ene_B,CUSMA_B,Total_B,Auto_E,...,Total_E,Auto_C,Alum_C,Steel_C,Cop_C,Lum_C,Ene_C,CUSMA_C,Total_C,geometry
0,2021S051610010001,0,1,1,0,2,0,16,16,0,...,791,0.0,1.0,1.0,0.0,5.0,0.0,579.0,579.0,POINT (-53.25419 47.86225)
1,2021S051610010002,0,0,0,0,0,0,2,2,0,...,10,6.0,8.0,8.0,1.0,3.0,12.0,86.0,86.0,POINT (-52.76054 47.72309)
2,2021S051610010003,0,1,1,0,4,0,9,9,0,...,49,0.0,1.0,2.0,0.0,8.0,36.0,258.0,258.0,POINT (-53.26649 47.73593)
3,2021S051610010004,0,1,1,1,0,2,7,7,0,...,67,6.0,9.0,8.0,1.0,4.0,24.0,107.0,107.0,POINT (-52.71312 47.62177)
4,2021S051610010005,2,1,1,0,0,0,5,5,5,...,29,6.0,9.0,8.0,1.0,4.0,29.0,122.0,122.0,POINT (-52.7703 47.64725)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-115.3713 67.80897)
5429,2021S051662080005,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-95.88322 68.64144)
5430,2021S051662080006,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-89.80822 68.53258)
5431,2021S051662080007,0,0,0,0,0,0,0,0,0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,POINT (-107.82111 67.68532)


In [25]:
perc_bytariffs = census_bytariffs.copy()

for tar in tars:
    tar_3 = f'{tar}_3'

    perc_bytariffs[tar_3] = (
        perc_bytariffs[f'{tar}_C']/perc_bytariffs['Census']
    )

perc_bytariffs = perc_bytariffs[['ADADGUID', 'Census'] + [f'{tar}_3' for tar in tars]]
perc_bytariffs = perc_bytariffs.fillna(0)
perc_bytariffs['CUSMA_3'] = perc_bytariffs['CUSMA_3'].clip(upper=1)
perc_bytariffs['Total_3'] = perc_bytariffs['Total_3'].clip(upper=1)
perc_bytariffs

,ADADGUID,Census,Auto_3,Alum_3,Steel_3,Cop_3,Lum_3,Ene_3,CUSMA_3,Total_3
0,2021S051610010001,3715.0,0.000000,0.000269,0.000269,0.000000,0.001346,0.000000,0.155855,0.155855
1,2021S051610010002,2120.0,0.002830,0.003774,0.003774,0.000472,0.001415,0.005660,0.040566,0.040566
2,2021S051610010003,4530.0,0.000000,0.000221,0.000442,0.000000,0.001766,0.007947,0.056954,0.056954
3,2021S051610010004,5145.0,0.001166,0.001749,0.001555,0.000194,0.000777,0.004665,0.020797,0.020797
4,2021S051610010005,5190.0,0.001156,0.001734,0.001541,0.000193,0.000771,0.005588,0.023507,0.023507
...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,510.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5429,2021S051662080005,465.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5430,2021S051662080006,280.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5431,2021S051662080007,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [26]:
choropleth = choro_gdf.merge(perc_bytariffs, on='ADADGUID', how='right')
choropleth = choropleth[['ADADGUID', 'All_Businesses', 'All_Employees', 'Census'] + [f'{tar}_1' for tar in tars] + [f'{tar}_2' for tar in tars] + [f'{tar}_3' for tar in tars] + ['geometry']]
choropleth = choropleth.to_crs('EPSG:4326')
choropleth

,ADADGUID,All_Businesses,All_Employees,Census,Auto_1,Alum_1,Steel_1,Cop_1,Lum_1,Ene_1,...,Total_2,Auto_3,Alum_3,Steel_3,Cop_3,Lum_3,Ene_3,CUSMA_3,Total_3,geometry
0,2021S051610010001,219,2910,3715.0,0.000000,0.004566,0.004566,0.000000,0.009132,0.000000,...,0.271821,0.000000,0.000269,0.000269,0.000000,0.001346,0.000000,0.155855,0.155855,"MULTIPOLYGON (((-53.51451 47.69912, -53.51464 ..."
1,2021S051610010002,66,394,2120.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025381,0.002830,0.003774,0.003774,0.000472,0.001415,0.005660,0.040566,0.040566,"POLYGON ((-52.78543 47.80961, -52.78542 47.809..."
2,2021S051610010003,288,4186,4530.0,0.000000,0.003472,0.003472,0.000000,0.013889,0.000000,...,0.011706,0.000000,0.000221,0.000442,0.000000,0.001766,0.007947,0.056954,0.056954,"MULTIPOLYGON (((-53.12645 47.81702, -53.12663 ..."
3,2021S051610010004,438,9064,5145.0,0.000000,0.002283,0.002283,0.002283,0.000000,0.004566,...,0.007392,0.001166,0.001749,0.001555,0.000194,0.000777,0.004665,0.020797,0.020797,"POLYGON ((-52.66902 47.66588, -52.66898 47.665..."
4,2021S051610010005,203,1441,5190.0,0.009852,0.004926,0.004926,0.000000,0.000000,0.000000,...,0.020125,0.001156,0.001734,0.001541,0.000193,0.000771,0.005588,0.023507,0.023507,"POLYGON ((-52.73992 47.69568, -52.74026 47.694..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5428,2021S051662080004,11,356,510.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-115.34503 67.89695, -115.3453..."
5429,2021S051662080005,13,414,465.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-95.82942 68.59941, -95.82932 ..."
5430,2021S051662080006,8,295,280.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-89.84909 68.53759, -89.84931 ..."
5431,2021S051662080007,0,0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,"MULTIPOLYGON (((-107.9628 67.7012, -107.9628 6..."


In [27]:
choropleth.to_file('choropleth.geojson', driver='GeoJSON')
choropleth.to_file('choropleth.shp', driver='ESRI Shapefile')

C:\Users\mkbs_\AppData\Local\Temp\ipykernel_12344\4259686084.py:2: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  choropleth.to_file('choropleth.shp', driver='ESRI Shapefile')
C:\Users\mkbs_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'All_Businesses' to 'All_Busine'
  ogr_write(
C:\Users\mkbs_\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\pyogrio\raw.py:723: RuntimeWarning: Normalized/laundered field name: 'All_Employees' to 'All_Employ'
  ogr_write(


# CSV Trails

for double-checking purposes

In [28]:
business_grouped.drop(columns='geometry').to_csv('trail.csv', index=False)
business_filter.drop(columns='geometry').to_csv('trail2.csv', index=False)
business_census.to_csv('trail3.csv', index=False)

In [29]:
choro_cols.drop(columns='geometry').to_csv('trail4.csv', index=False)

In [30]:
jobs.to_csv('trail5.csv')
jobs_rate.to_csv('trail6.csv')
adjusted_jobs.to_csv('trail7.csv')